In [1]:
import os
import pickle
import logging

import pandas as pd
import numpy as np

from hbmep.config import Config
from hbmep.model.utils import Site as site

from paper.utils import setup_logging
from models import NonHierarchicalBayesianModel
from constants import (
    TOML_PATH,
    BUILD_DIR
)


In [2]:
src = "/home/vishu/data/hbmep-processed/rat/J_RCML/data.csv"

df = pd.read_csv(src)
df.shape


(7676, 44)

In [3]:
config = Config(toml_path=TOML_PATH)
model = NonHierarchicalBayesianModel(config=config)


In [4]:
combinations = (
    df[model.features]
    .apply(tuple, axis=1)
    .unique()
    .tolist()
)
combinations = sorted(combinations)
len(combinations)


150

In [5]:
posterior_samples = {site.a: None}

for c in combinations:
    subject, position = c
    dir = os.path.join(
        BUILD_DIR,
        f"sub__{subject}",
        f"pos__{position}"
    )

    response_samples = None

    for response in model.response:
        src = os.path.join(
            dir, f"resp__{response}", model.NAME, "inference.pkl"
        )
        with open(src, "rb") as f:
            _, _, samples = pickle.load(f)

        for u in posterior_samples.keys():
            site_samples = samples[u].reshape(-1,)
            site_samples = site_samples[:, None, None]

            if response_samples is None:
                response_samples = site_samples
            else:
                response_samples = np.concatenate(
                    [response_samples, site_samples],
                    axis=-1
                )

    if posterior_samples[site.a] is None:
        posterior_samples[site.a] = response_samples
    else:
        posterior_samples[site.a] = np.concatenate(
            [posterior_samples[site.a], response_samples],
            axis=-2
        )


In [6]:
dest = os.path.join(BUILD_DIR, "inference.pkl")

with open(dest, "wb") as f:
    pickle.dump((combinations, posterior_samples,), f)
